In [24]:
 # 確保文字編碼正確
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# 查看 GPU 狀態
!nvidia-smi

# 安裝 ultralytics（YOLOv8 套件）
!pip install flask flask-ngrok ultralytics

# 匯入所需模組
from ultralytics import YOLO
import os
from google.colab import drive, files

Fri Jun 20 07:13:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   73C    P0             29W /   70W |     334MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# 1) Install dependencies
!pip install flask pyngrok ultralytics

# 2) Import what we need
from flask import Flask, request, jsonify
from pyngrok import ngrok
ngrok.set_auth_token("2ylKUgkeXSZRti20YAm3CI1PeD0_7YrVPqxqA9Up8ZeeAf41Y")
from ultralytics import YOLO
import os

# 3) Spin up the model
model = YOLO("/content/drive/MyDrive/Colab Notebooks/medicine/best.pt")

# 4) Create Flask app
app = Flask(__name__)

# 5) Open an ngrok tunnel on port 5000 and print the URL
public_url = ngrok.connect(5000).public_url
print(f"🚀 * Public scan endpoint: {public_url}/scan")

# 6) Define the /scan route
@app.route('/scan', methods=['POST'])
def scan():
    file = request.files.get('image')
    if not file:
        return jsonify({'error': 'no image provided'}), 400

    save_path = os.path.join('/content', file.filename)
    file.save(save_path)

    results = model(save_path, imgsz=1280)
    matched = []
    for r in results:
        for cls, conf in zip(r.boxes.cls, r.boxes.conf):
            matched.append({
                'name': model.names[int(cls)],
                'confidence': float(conf),
            })

    return jsonify({'matchedDrugs': matched})

# 7) Run the Flask app
if __name__ == '__main__':
    app.run(port=5000)


🚀 * Public scan endpoint: https://770a-34-82-205-177.ngrok-free.app/scan
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
# 掛載 Google 雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

# 資料集路徑（你的 data.yaml）
#yaml_path = "/content/drive/MyDrive/Colab_Notebooks/medicine/data/data.yaml"
#如如
#yaml_path = "/content/drive/MyDrive/Colab Notebooks/medicine/data/data.yaml"
yaml_path = "/content/drive/MyDrive/Colab Notebooks/medicine/data/data.yaml"
#德成
if not os.path.exists(yaml_path):
    raise FileNotFoundError(f"找不到 {yaml_path}，請確認路徑正確！")

# 模型儲存路徑
model_save_path = "/content/drive/MyDrive/Colab_Notebooks/medicine/best.pt"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 只需一次，重新連線後不用執行
# 載入預訓練 YOLOv8 模型
#model = YOLO("yolov8n.pt")  # 可以改成 yolov8s.pt / yolov8m.pt 等其他版本

# 開始訓練
#model.train(data=yaml_path, epochs=100, batch=16, imgsz=1280)

# 將訓練好的模型儲存到 Google Drive
#!cp runs/detect/train/weights/best.pt "{model_save_path}"

# 驗證模型表現
#model = YOLO(model_save_path)
#metrics = model.val()
#print(metrics)

###測試

In [23]:
# 讀取儲存在 Google Drive 的訓練完成模型
model_path = "/content/drive/MyDrive/Colab Notebooks/medicine/best.pt"
model = YOLO(model_path)

# 上傳圖片進行推論
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 推論（可調整 imgsz）
results = model(f"/content/{filename}", imgsz=1280)

# 顯示與儲存圖片
results[0].show()
results[0].save(filename=f"/content/detected_{filename}")

# 解析預測結果
for result in results:
    for box in result.boxes:
        class_id = int(box.cls)
        confidence = float(box.conf)
        label = model.names[class_id]
        xyxy = box.xyxy[0].tolist()

        print(f"偵測到藥物: {label}，機率: {confidence:.2%}，座標: {xyxy}")

# 下載標註後圖片
files.download(f"/content/detected_{filename}")

TypeError: 'NoneType' object is not subscriptable